In [1]:
import json

# read jsonl file from disk
with open('llama3-70b-solution-last.jsonl', 'r') as f:
    jsonl_lines = f.readlines()

import re


def extract_answer(data):
    # Step 1: Find the location of the "answer" keyword
    answer_start = data.find('"answer')
    if answer_start == -1:
        return None  # If "answer" is not found, return None

    # Step 2: Find the start of the value after "answer"
    answer_start = data.find(':', answer_start)  # Find the colon after "answer"
    if answer_start == -1:
        return None  # If the colon is not found, return None

    # Step 3: Find the first occurrence of "\ndata" after the colon
    data_end = data.find("\ndata", answer_start)
    if data_end == -1:
        return None  # If "\ndata" is not found, return None

    # Step 4: Extract the value between the colon and the first "\ndata"
    answer_value = data[answer_start + 1:data_end].strip().strip('"')

    return answer_value

# Function to extract Problem ID and Answer
def extract_problem_and_answer(jsonl_lines):
    extracted_data = []
    for line in jsonl_lines:
        # Try to load the JSON data from the line
        try:
            # Assuming the first key is always the problem id
            data = json.loads(line)
            problem_id = list(data.keys())[0]  # Get the first key as problem ID
            problem_data = data[problem_id]  # Get the associated data as a string

            # Find and extract the JSON object in the 'data' field
            start_index = problem_data.find('{')
            end_index = problem_data.rfind('}') + 1

            answer = extract_answer(problem_data[start_index:end_index])

            # Construct the JSONL format output
            extracted_data.append({problem_id: {"answer": str(answer)}})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    
    return extracted_data

# Extract data from the JSONL lines
result = extract_problem_and_answer(jsonl_lines)

# Output the extracted problem IDs and answers in JSONL format
with open('clean/output-last.jsonl', 'w') as f:
    for item in result:
        f.write(json.dumps(item) + '\n')


In [19]:
import json

# read jsonl file from disk
with open('gpt-4-o1-preview-solution.jsonl', 'r') as f:
    jsonl_lines = f.readlines()

import re


def extract_answer(data):
    # Step 1: Find the location of the "answer" keyword
    answer_start = data.find('answer')
    if answer_start == -1:
        print(f'something went wrong:{data}')
        return None  # If "answer" is not found, return None

    # Step 2: Find the start of the value after "answer"
    answer_start = data.find(':', answer_start)  # Find the colon after "answer"
    if answer_start == -1:
        print(f'something went wrong:{data}')
        return None  # If the colon is not found, return None

    # Step 3: Find the first occurrence of "\ndata" after the colon
    data_end = data.find("}", answer_start)
    if data_end == -1:
        print(f'something went wrong:{data}')
        return None  

    # Step 4: Extract the value between the colon and the first "\ndata"
    answer_value = data[answer_start + 1:data_end].strip().strip('"')

    return answer_value

# Function to extract Problem ID and Answer
def extract_problem_and_answer(jsonl_lines):
    extracted_data = []
    for line in jsonl_lines:
        # Assuming the first key is always the problem id
        data = json.loads(line)
        problem_id = list(data.keys())[0]  # Get the first key as problem ID
        problem_data = data[problem_id]  # Get the associated data as a string

        answer = extract_answer(problem_data)

        # Construct the JSONL format output
        extracted_data.append({problem_id: {"answer": str(answer)}})
    
    
    return extracted_data

# Extract data from the JSONL lines
result = extract_problem_and_answer(jsonl_lines)

# Output the extracted problem IDs and answers in JSONL format
with open('clean/gpt-4-o1-preview-solution-clean.jsonl', 'w') as f:
    for item in result:
        f.write(json.dumps(item) + '\n')


something went wrong:```json
{
    "a": -3,
    "b": 4
}
```


In [11]:
import json
# Function to extract and clean data
def clean_data(input_data):
    cleaned_data = []
    
    for line in input_data:
        entry = json.loads(line)
        for problem_id, text in entry.items():
            # Extract json block from the text
            start = text.find('\"answer\":')
            end = text.find('\n', start + 1)
            json_text = text[start + len('```json'):end].strip()
            try:
                json_data = json.loads(json_text)
                cleaned_data.append({problem_id: json_data})
            except json.JSONDecodeError:
                print(f'something went wrong:{problem_id}')
                print(json_text)
           
    
    return cleaned_data

In [5]:
# read jsonl file from disk
import json
with open('gpt-4-0125-preview-solution-new.jsonl', 'r') as f:
    jsonl_lines = f.readlines()


In [12]:
aaa = json.loads(jsonl_lines[0])
bbb = json.loads(aaa['Problem_1'])
bbb['answer']

8

In [34]:
# Function to properly escape backslashes in JSON strings
def escape_backslashes(text):
    # Replace single backslashes with double backslashes
    return text.replace("\\", "\\\\")

In [44]:
import json

# Input and output file paths
input_file = '/root/Xiangpeng/odyssey-math/jsonl/gpt-4-0125-preview-solution-new.jsonl'
output_file = 'clean/gpt-4-0125-preview-solution-clean-new.jsonl'

# Open the input file for reading and the output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Parse the JSON data
        data = json.loads(line)
        
        # Extract only the answer and restructure the JSON
        for key, value in data.items():
            try:
                # Fix escape sequences in the nested JSON string
                fixed_value = escape_backslashes(value)
                value = json.loads(fixed_value)
                cleaned_data = {key: {"answer": value["answer"]}}
                # Write the cleaned JSON back to the output file
                outfile.write(json.dumps(cleaned_data) + "\n")
            except:
                print(key)
       


Problem_7
Problem_24
Problem_71
Problem_85
Problem_245
Problem_261
Problem_354
Problem_384


In [53]:
import json
import re

# Input and output file paths
input_file = '/root/Xiangpeng/odyssey-math/jsonl/gpt-4-1106-preview.jsonl'
output_file = 'clean/gpt-4-1106-preview-solution-clean-new.jsonl'

# Open the input file for reading and the output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Parse the JSON data
        data = json.loads(line)
        
        # Extract only the answer and restructure the JSON
        for key, value in data.items():
            
            # Find the start position of the last "answer":
            start_pos = value.rfind('"answer":')
            if start_pos != -1:
                end_pos = value.find('}', start_pos)
                captured_answer = value[start_pos:end_pos+1]
                
                cleaned_data = {key: {"answer": captured_answer.strip()}}    
                outfile.write(json.dumps(cleaned_data) + "\n")
            else:
                print(f"No answer field found in the {key}")
                cleaned_data = {key: {"answer": None}}    
                outfile.write(json.dumps(cleaned_data) + "\n")



No answer field found in the Problem_10
No answer field found in the Problem_16
No answer field found in the Problem_20
No answer field found in the Problem_38
No answer field found in the Problem_57
No answer field found in the Problem_119
No answer field found in the Problem_139
No answer field found in the Problem_146
No answer field found in the Problem_293


In [54]:
import json
import re

# Input and output file paths
input_file = '/root/Xiangpeng/odyssey-math/jsonl/gpt-3.5-turbo-0125.jsonl'
output_file = 'clean/gpt-3.5-turbo-0125-solution-clean-new.jsonl'

# Open the input file for reading and the output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Parse the JSON data
        data = json.loads(line)
        
        # Extract only the answer and restructure the JSON
        for key, value in data.items():
            
            # Find the start position of the last "answer":
            start_pos = value.rfind('"answer":')
            if start_pos != -1:
                end_pos = value.find('}', start_pos)
                captured_answer = value[start_pos:end_pos+1]
                
                cleaned_data = {key: {"answer": captured_answer.strip()}}    
                outfile.write(json.dumps(cleaned_data) + "\n")
            else:
                print(f"No answer field found in the {key}")
                cleaned_data = {key: {"answer": None}}    
                outfile.write(json.dumps(cleaned_data) + "\n")



No answer field found in the Problem_3
No answer field found in the Problem_8
No answer field found in the Problem_9
No answer field found in the Problem_10
No answer field found in the Problem_12
No answer field found in the Problem_13
No answer field found in the Problem_14
No answer field found in the Problem_15
No answer field found in the Problem_17
No answer field found in the Problem_19
No answer field found in the Problem_20
No answer field found in the Problem_24
No answer field found in the Problem_25
No answer field found in the Problem_27
No answer field found in the Problem_28
No answer field found in the Problem_32
No answer field found in the Problem_34
No answer field found in the Problem_37
No answer field found in the Problem_41
No answer field found in the Problem_42
No answer field found in the Problem_44
No answer field found in the Problem_46
No answer field found in the Problem_47
No answer field found in the Problem_50
No answer field found in the Problem_51
No 